In [289]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

player_attr_df = pd.read_csv('../data/raw/Player_Attributes.csv')
player_df = pd.read_csv('../data/raw/Player.csv')

In [290]:
print("Player Attributes dataset:")
print(player_attr_df.shape)
player_attr_df.head()

Player Attributes dataset:
(183978, 42)


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [291]:
player_attr_df['date'] = pd.to_datetime(player_attr_df['date'])
player_attr_df['player_api_id'] = pd.to_numeric(player_attr_df['player_api_id'], errors='coerce').astype('Int64')
player_attr_df.head(3)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0


In [292]:
player_attr_df.tail(3)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
183975,183976,102359,39902,2008-08-30,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0
183977,183978,102359,39902,2007-02-22,80.0,81.0,right,medium,low,74.0,...,88.0,53.0,38.0,32.0,30.0,9.0,9.0,78.0,7.0,15.0


In [293]:
player_attr_df = player_attr_df.drop(['player_fifa_api_id'], axis=1)
player_attr_df = player_attr_df.drop(['id'], axis=1)
player_attr_df = player_attr_df.set_index(['player_api_id', 'date'])
player_attr_df

overall_rating  potential preferred_foot  \
player_api_id date                                                   
505942        2016-02-18            67.0       71.0          right   
              2015-11-19            67.0       71.0          right   
              2015-09-21            62.0       66.0          right   
              2015-03-20            61.0       65.0          right   
              2007-02-22            61.0       65.0          right   
...                                  ...        ...            ...   
39902         2009-08-30            83.0       85.0          right   
              2009-02-22            78.0       80.0          right   
              2008-08-30            77.0       80.0          right   
              2007-08-30            78.0       81.0          right   
              2007-02-22            80.0       81.0          right   

                         attacking_work_rate defensive_work_rate  crossing  \
player_api_id date                                                           
505942        2016-02-18              medium              medium      49.0   
              2015-11-19              medium              medium      49.0   
              2015-09-21              medium              medium      49.0   
              2015-03-20              medium              medium      48.0   
              2007-02-22              medium              medium      48.0   
...                                      ...                 ...       ...   
39902         2009-08-30              medium                 low      84.0   
              2009-02-22              medium                 low      74.0   
              2008-08-30              medium                 low      74.0   
              2007-08-30              medium                 low      74.0   
              2007-02-22              medium                 low      74.0   

                          finishing  heading_accuracy  short_passing  volleys  \
player_api_id date                                                              
505942        2016-02-18       44.0              71.0           61.0     44.0   
              2015-11-19       44.0              71.0           61.0     44.0   
              2015-09-21       44.0              71.0           61.0     44.0   
              2015-03-20       43.0              70.0           60.0     43.0   
              2007-02-22       43.0              70.0           60.0     43.0   
...                             ...               ...            ...      ...   
39902         2009-08-30       77.0              59.0           89.0     77.0   
              2009-02-22       76.0              53.0           84.0     77.0   
              2008-08-30       71.0              53.0           84.0     77.0   
              2007-08-30       64.0              57.0           86.0     77.0   
              2007-02-22       68.0              57.0           88.0     77.0   

                          ...  vision  penalties  marking  standing_tackle  \
player_api_id date        ...                                                
505942        2016-02-18  ...    54.0       48.0     65.0             69.0   
              2015-11-19  ...    54.0       48.0     65.0             69.0   
              2015-09-21  ...    54.0       48.0     65.0             66.0   
              2015-03-20  ...    53.0       47.0     62.0             63.0   
              2007-02-22  ...    53.0       47.0     62.0             63.0   
...                       ...     ...        ...      ...              ...   
39902         2009-08-30  ...    88.0       83.0     22.0             31.0   
              2009-02-22  ...    88.0       70.0     32.0             31.0   
              2008-08-30  ...    88.0       70.0     32.0             31.0   
              2007-08-30  ...    88.0       53.0     28.0             32.0   
              2007-02-22  ...    88.0       53.0     38.0             32.0   

                          slidi

In [294]:
num_nulls = player_attr_df.isna().sum()
print(f"Number of NaNs in each column:\n{num_nulls}")

Number of NaNs in each column:
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    6869
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle         836
sliding_tackle         2713
gk_diving               836
gk_handling      

In [295]:
player_attr_df.dropna(inplace=True)
player_attr_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 177037 entries, (np.int64(505942), Timestamp('2016-02-18 00:00:00')) to (np.int64(39902), Timestamp('2007-02-22 00:00:00'))
Data columns (total 38 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   overall_rating       177037 non-null  float64
 1   potential            177037 non-null  float64
 2   preferred_foot       177037 non-null  object 
 3   attacking_work_rate  177037 non-null  object 
 4   defensive_work_rate  177037 non-null  object 
 5   crossing             177037 non-null  float64
 6   finishing            177037 non-null  float64
 7   heading_accuracy     177037 non-null  float64
 8   short_passing        177037 non-null  float64
 9   volleys              177037 non-null  float64
 10  dribbling            177037 non-null  float64
 11  curve                177037 non-null  float64
 12  free_kick_accuracy   177037 non-null  float64
 13  long_passing         17

In [296]:
# unique_count = player_attr_df.index.get_level_values('player_api_id').nunique()
# print(f"Number of unique players:\n{unique_count}")


In [297]:
#organize data in numerical order by api id and then chronologically by date
player_attr_df = player_attr_df.sort_index(level=['player_api_id', 'date'])
player_attr_df

overall_rating  potential preferred_foot  \
player_api_id date                                                   
2625          2007-02-22            63.0       64.0          right   
              2007-08-30            63.0       64.0          right   
              2008-08-30            60.0       64.0          right   
              2010-08-30            60.0       64.0          right   
              2011-02-22            59.0       63.0          right   
...                                  ...        ...            ...   
750435        2016-02-04            56.0       70.0          right   
              2016-02-11            56.0       70.0          right   
              2016-04-14            60.0       74.0          right   
750584        2007-02-22            58.0       66.0          right   
              2016-05-19            58.0       66.0          right   

                         attacking_work_rate defensive_work_rate  crossing  \
player_api_id date                                                           
2625          2007-02-22                high              medium      48.0   
              2007-08-30                high              medium      48.0   
              2008-08-30                high              medium      48.0   
              2010-08-30                high              medium      48.0   
              2011-02-22                high              medium      52.0   
...                                      ...                 ...       ...   
750435        2016-02-04              medium                 low      35.0   
              2016-02-11              medium                 low      35.0   
              2016-04-14              medium                 low      35.0   
750584        2007-02-22              medium              medium      47.0   
              2016-05-19              medium              medium      47.0   

                          finishing  heading_accuracy  short_passing  volleys  \
player_api_id date                                                              
2625          2007-02-22       48.0              47.0           64.0     38.0   
              2007-08-30       48.0              47.0           64.0     38.0   
              2008-08-30       48.0              47.0           64.0     38.0   
              2010-08-30       48.0              47.0           64.0     38.0   
              2011-02-22       47.0              46.0           63.0     37.0   
...                             ...               ...            ...      ...   
750435        2016-02-04       57.0              55.0           60.0     52.0   
              2016-02-11       57.0              55.0           60.0     52.0   
              2016-04-14       57.0              55.0           60.0     52.0   
750584        2007-02-22       39.0              53.0           60.0     37.0   
              2016-05-19       39.0              53.0           60.0     37.0   

                          ...  vision  penalties  marking  standing_tackle  \
player_api_id date        ...                                                
2625          2007-02-22  ...    56.0       46.0     64.0             66.0   
              2007-08-30  ...    56.0       46.0     64.0             66.0   
              2008-08-30  ...    56.0       46.0     64.0             66.0   
              2010-08-30  ...    56.0       69.0     64.0             66.0   
              2011-02-22  ...    55.0       66.0     63.0             63.0   
...                       ...     ...        ...      ...              ...   
750435        2016-02-04  ...    58.0       61.0     18.0             19.0   
              2016-02-11  ...    58.0       61.0     18.0             19.0   
              2016-04-14  ...    58.0       61.0     18.0             19.0   
750584        2007-02-22  ...    47.0       52.0     56.0             66.0   
              2016-05-19  ...    47.0       52.0     56.0             66.0   

                          slidi

# Encoding categorical variables (3 total)

# preferred_foot
We mapped right to 1 and 0 to left for preferred_foot as a form of one-hot encoding.

In [298]:
player_attr_df['preferred_foot'] = player_attr_df['preferred_foot'].map({'right': 1, 'left': 0})
player_attr_df


overall_rating  potential  preferred_foot  \
player_api_id date                                                    
2625          2007-02-22            63.0       64.0               1   
              2007-08-30            63.0       64.0               1   
              2008-08-30            60.0       64.0               1   
              2010-08-30            60.0       64.0               1   
              2011-02-22            59.0       63.0               1   
...                                  ...        ...             ...   
750435        2016-02-04            56.0       70.0               1   
              2016-02-11            56.0       70.0               1   
              2016-04-14            60.0       74.0               1   
750584        2007-02-22            58.0       66.0               1   
              2016-05-19            58.0       66.0               1   

                         attacking_work_rate defensive_work_rate  crossing  \
player_api_id date                                                           
2625          2007-02-22                high              medium      48.0   
              2007-08-30                high              medium      48.0   
              2008-08-30                high              medium      48.0   
              2010-08-30                high              medium      48.0   
              2011-02-22                high              medium      52.0   
...                                      ...                 ...       ...   
750435        2016-02-04              medium                 low      35.0   
              2016-02-11              medium                 low      35.0   
              2016-04-14              medium                 low      35.0   
750584        2007-02-22              medium              medium      47.0   
              2016-05-19              medium              medium      47.0   

                          finishing  heading_accuracy  short_passing  volleys  \
player_api_id date                                                              
2625          2007-02-22       48.0              47.0           64.0     38.0   
              2007-08-30       48.0              47.0           64.0     38.0   
              2008-08-30       48.0              47.0           64.0     38.0   
              2010-08-30       48.0              47.0           64.0     38.0   
              2011-02-22       47.0              46.0           63.0     37.0   
...                             ...               ...            ...      ...   
750435        2016-02-04       57.0              55.0           60.0     52.0   
              2016-02-11       57.0              55.0           60.0     52.0   
              2016-04-14       57.0              55.0           60.0     52.0   
750584        2007-02-22       39.0              53.0           60.0     37.0   
              2016-05-19       39.0              53.0           60.0     37.0   

                          ...  vision  penalties  marking  standing_tackle  \
player_api_id date        ...                                                
2625          2007-02-22  ...    56.0       46.0     64.0             66.0   
              2007-08-30  ...    56.0       46.0     64.0             66.0   
              2008-08-30  ...    56.0       46.0     64.0             66.0   
              2010-08-30  ...    56.0       69.0     64.0             66.0   
              2011-02-22  ...    55.0       66.0     63.0             63.0   
...                       ...     ...        ...      ...              ...   
750435        2016-02-04  ...    58.0       61.0     18.0             19.0   
              2016-02-11  ...    58.0       61.0     18.0             19.0   
              2016-04-14  ...    58.0       61.0     18.0             19.0   
750584        2007-02-22  ...    47.0       52.0     56.0             66.0   
              2016-05-19  ...    47.0       52.0     56.0             66.0   

                  

## attacking_work_rate
We imputed the nonsense descriptions in 'attacking_work_rate' that were not "high," "medium," or "low" with "medium" because "medium" was the most frequent value, constituting more than two thirds of the rows. 

In [299]:
unique_values = player_attr_df['attacking_work_rate'].unique()
#print(f"Unique values in 'attacking_work_rate': {unique_values}")
value_counts = player_attr_df['attacking_work_rate'].value_counts()
player_attr_df['attacking_work_rate'] = player_attr_df['attacking_work_rate'].astype(str)
# Replace any value not 'high', 'medium', or 'low' with 'medium' (as imputed previously)
player_attr_df['attacking_work_rate'] = player_attr_df['attacking_work_rate'].apply(
    lambda x: x if x in ['high', 'medium', 'low'] else 'medium'
)
# Now map to integers
player_attr_df['attacking_work_rate'] = player_attr_df['attacking_work_rate'].map({'high': 2, 'medium': 1, 'low': 0}).astype(int)


In [300]:
unique_values = player_attr_df['defensive_work_rate'].unique()
player_attr_df['defensive_work_rate'] = player_attr_df['defensive_work_rate'].astype(str)
print(player_attr_df['defensive_work_rate'].value_counts())
player_attr_df['defensive_work_rate'] = player_attr_df['defensive_work_rate'].apply(
    lambda x: x if x in ['high', 'medium', 'low'] else 'medium'
)
# Now map to integers
player_attr_df['defensive_work_rate'] = player_attr_df['defensive_work_rate'].map({'high': 2, 'medium': 1, 'low': 0}).astype(int)
player_attr_df
print(player_attr_df['defensive_work_rate'].value_counts())
player_attr_df

defensive_work_rate
medium    130764
high       26972
low        18425
ormal        317
es            94
2             89
tocky         86
ean           78
1             44
6             34
5             28
4             27
3             23
0             20
9             17
7             14
8              5
Name: count, dtype: int64
defensive_work_rate
1    131640
2     26972
0     18425
Name: count, dtype: int64


overall_rating  potential  preferred_foot  \
player_api_id date                                                    
2625          2007-02-22            63.0       64.0               1   
              2007-08-30            63.0       64.0               1   
              2008-08-30            60.0       64.0               1   
              2010-08-30            60.0       64.0               1   
              2011-02-22            59.0       63.0               1   
...                                  ...        ...             ...   
750435        2016-02-04            56.0       70.0               1   
              2016-02-11            56.0       70.0               1   
              2016-04-14            60.0       74.0               1   
750584        2007-02-22            58.0       66.0               1   
              2016-05-19            58.0       66.0               1   

                          attacking_work_rate  defensive_work_rate  crossing  \
player_api_id date                                                             
2625          2007-02-22                    2                    1      48.0   
              2007-08-30                    2                    1      48.0   
              2008-08-30                    2                    1      48.0   
              2010-08-30                    2                    1      48.0   
              2011-02-22                    2                    1      52.0   
...                                       ...                  ...       ...   
750435        2016-02-04                    1                    0      35.0   
              2016-02-11                    1                    0      35.0   
              2016-04-14                    1                    0      35.0   
750584        2007-02-22                    1                    1      47.0   
              2016-05-19                    1                    1      47.0   

                          finishing  heading_accuracy  short_passing  volleys  \
player_api_id date                                                              
2625          2007-02-22       48.0              47.0           64.0     38.0   
              2007-08-30       48.0              47.0           64.0     38.0   
              2008-08-30       48.0              47.0           64.0     38.0   
              2010-08-30       48.0              47.0           64.0     38.0   
              2011-02-22       47.0              46.0           63.0     37.0   
...                             ...               ...            ...      ...   
750435        2016-02-04       57.0              55.0           60.0     52.0   
              2016-02-11       57.0              55.0           60.0     52.0   
              2016-04-14       57.0              55.0           60.0     52.0   
750584        2007-02-22       39.0              53.0           60.0     37.0   
              2016-05-19       39.0              53.0           60.0     37.0   

                          ...  vision  penalties  marking  standing_tackle  \
player_api_id date        ...                                                
2625          2007-02-22  ...    56.0       46.0     64.0             66.0   
              2007-08-30  ...    56.0       46.0     64.0             66.0   
              2008-08-30  ...    56.0       46.0     64.0             66.0   
              2010-08-30  ...    56.0       69.0     64.0             66.0   
              2011-02-22  ...    55.0       66.0     63.0             63.0   
...                       ...     ...        ...      ...              ...   
750435        2016-02-04  ...    58.0       61.0     18.0             19.0   
              2016-02-11  ...    58.0       61.0     18.0             19.0   
              2016-04-14  ...    58.0       61.0     18.0             19.0   
750584        2007-02-22  ...    47.0       52.0     56.0             66.0   
              2016-05-19  ...    47.0       52.0     56.0             6

In [301]:
player_attr_df.to_csv('player_attributes_cleaned.csv', index=False)